In [11]:
import json
from pprint import pprint

In [12]:
# Read data

with open('train_new.json') as f:
    data1=json.load(f)

data1.keys()
#pprint(data)

dict_keys(['content', 'date', 'tags', 'title', 'url'])

In [13]:
with open('test_new.json') as f:
    data2=json.load(f)

data1.keys()
#pprint(data)

dict_keys(['content', 'date', 'tags', 'title', 'url'])

In [14]:
#Prepare text data

import pandas as pd
set=[]
for k,v in data1['content'].items():
    #print(k)
    #print(v)
    set.append(v)
    
import pandas as pd

for k,v in data2['content'].items():
    #print(k)
    #print(v)
    set.append(v)
    
df1=pd.DataFrame(set)
print(df1.head())
print(df1.shape)

                                                   0
0  Approximately 1.6 million model-year 2015-18 F...
1  Approximately 192,000 model-year 2016-18 Toyot...
2  Nobody likes when things don't work the way th...
3  Approximately 19,400 model-year 2012 Toyota Av...
4  Approximately 8,400 model-year 2017-18 Volkswa...
(2815, 1)


In [15]:
# df1.columns=['a']

In [16]:
# df1['a']

0       Approximately 1.6 million model-year 2015-18 F...
1       Approximately 192,000 model-year 2016-18 Toyot...
2       Nobody likes when things don't work the way th...
3       Approximately 19,400 model-year 2012 Toyota Av...
4       Approximately 8,400 model-year 2017-18 Volkswa...
5       Approximately 9,000 model-year 2018 Mercedes-B...
6       Approximately 12,800 model-year 2018 Jeep Rene...
7       Approximately 154,000 model-year 2018-19 Dodge...
8       Approximately 90 model-year 2018 Ford Edge and...
9       Approximately 49,000 model-year 2012-15 Ford F...
10      Approximately 470 model-year 2018 Mercedes-Ben...
11      Approximately 3,300 model-year 2018 Genesis G8...
12      Approximately 300 model-year 2019 Subaru Ascen...
13      Approximately 65 model-year 2018 Chrysler Paci...
14      12 model-year 2018 BMW 540d xDrive sedans\n Th...
15      's 2018 Volkswagen Atlas, a long-term test veh...
16      Nobody likes when things don't work the way th...
17      As the

In [17]:
#nltk.download()

In [ ]:
#Pre-process data

In [30]:
import nltk

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

from nltk.corpus import stopwords
mystopwords=stopwords.words("English") + ['one', 'become', 'get', 'make', 'take', 'recall','said', 'say', 'could', 'nhtsa', 'n\'t', 'may', 'vehicle', 'car']
WNlemma = nltk.WordNetLemmatizer()

def pre_process(text):
    tokens = nltk.word_tokenize(text)
    tokens=[ WNlemma.lemmatize(t.lower()) for t in tokens]
    tokens=[ t for t in tokens if t not in mystopwords]
    tokens = [ t for t in tokens if len(t) >= 3 ]
    return(tokens)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\slegl\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\slegl\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\slegl\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [36]:
#Pre-process data

toks_train = df1['a'].apply(pre_process)

In [37]:
#!pip install gensim

In [42]:
import logging
import gensim 
from gensim import corpora

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

C:\Users\slegl\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [43]:
# Prepare a vocabulary dictionary.
dictionary = corpora.Dictionary(toks_train)
print(dictionary)

2018-10-16 11:55:36,393 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-10-16 11:55:36,842 : INFO : built Dictionary(25451 unique tokens: ['1.6', '2015-18', 'approximately', 'area', 'b-pillar']...) from 2815 documents (total 516136 corpus positions)


Dictionary(25451 unique tokens: ['1.6', '2015-18', 'approximately', 'area', 'b-pillar']...)


In [44]:
type(dictionary)

gensim.corpora.dictionary.Dictionary

In [45]:
dtm_train = [dictionary.doc2bow(d) for d in toks_train ]

In [46]:
# Build an LDA model for 3 topics out of the DTM
%time lda = gensim.models.ldamodel.LdaModel(dtm_train, num_topics = 5,  alpha='auto',id2word = dictionary)
#chunksize (int, optional) – Number of documents to be used in each training chunk.
#passes (int, optional) – Number of passes through the corpus during training.
#chunksize=30,
#passes = 20

2018-10-16 11:55:51,589 : INFO : using autotuned alpha, starting with [0.2, 0.2, 0.2, 0.2, 0.2]
2018-10-16 11:55:51,593 : INFO : using symmetric eta at 0.2
2018-10-16 11:55:51,605 : INFO : using serial LDA version on this node
2018-10-16 11:55:51,638 : INFO : running online (single-pass) LDA training, 5 topics, 1 passes over the supplied corpus of 2815 documents, updating model once every 2000 documents, evaluating perplexity every 2815 documents, iterating 50x with a convergence threshold of 0.001000
2018-10-16 11:55:51,638 : WARNING : too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2018-10-16 11:55:51,638 : INFO : PROGRESS: pass 0, at document #2000/2815
2018-10-16 11:55:55,106 : INFO : optimized alpha [0.33097643, 0.30232117, 0.28734493, 0.29300487, 0.29629004]
2018-10-16 11:55:55,106 : INFO : merging changes from 2000 documents into a model of 2815 documents
2018-10-16 11:55:55,131 : INFO : topic #0 (0.331): 0

Wall time: 9.34 s


In [47]:
# Most probable words in each topic. 
lda.show_topics()

[(0,
  '0.011*"owner" + 0.010*"safety" + 0.009*"model" + 0.009*"affected" + 0.008*"ford" + 0.007*"issue" + 0.007*"dealer" + 0.006*"year" + 0.006*"number" + 0.005*"airbag"'),
 (1,
  '0.011*"model" + 0.009*"owner" + 0.009*"ford" + 0.009*"safety" + 0.008*"2015" + 0.007*"affected" + 0.006*"2014" + 0.006*"year" + 0.006*"dealer" + 0.006*"fuel"'),
 (2,
  '0.011*"safety" + 0.010*"honda" + 0.009*"takata" + 0.007*"affected" + 0.007*"inflator" + 0.007*"problem" + 0.007*"owner" + 0.005*"automaker" + 0.005*"customer" + 0.005*"company"'),
 (3,
  '0.009*"safety" + 0.008*"chrysler" + 0.008*"number" + 0.007*"model" + 0.007*"year" + 0.006*"owner" + 0.006*"company" + 0.005*"2014" + 0.005*"affected" + 0.005*"problem"'),
 (4,
  '0.009*"toyota" + 0.009*"owner" + 0.008*"safety" + 0.008*"problem" + 0.006*"model" + 0.006*"affected" + 0.006*"dealer" + 0.005*"service" + 0.005*"also" + 0.005*"company"')]

In [48]:
lda.show_topics(num_words=10)

[(0,
  '0.011*"owner" + 0.010*"safety" + 0.009*"model" + 0.009*"affected" + 0.008*"ford" + 0.007*"issue" + 0.007*"dealer" + 0.006*"year" + 0.006*"number" + 0.005*"airbag"'),
 (1,
  '0.011*"model" + 0.009*"owner" + 0.009*"ford" + 0.009*"safety" + 0.008*"2015" + 0.007*"affected" + 0.006*"2014" + 0.006*"year" + 0.006*"dealer" + 0.006*"fuel"'),
 (2,
  '0.011*"safety" + 0.010*"honda" + 0.009*"takata" + 0.007*"affected" + 0.007*"inflator" + 0.007*"problem" + 0.007*"owner" + 0.005*"automaker" + 0.005*"customer" + 0.005*"company"'),
 (3,
  '0.009*"safety" + 0.008*"chrysler" + 0.008*"number" + 0.007*"model" + 0.007*"year" + 0.006*"owner" + 0.006*"company" + 0.005*"2014" + 0.005*"affected" + 0.005*"problem"'),
 (4,
  '0.009*"toyota" + 0.009*"owner" + 0.008*"safety" + 0.008*"problem" + 0.006*"model" + 0.006*"affected" + 0.006*"dealer" + 0.005*"service" + 0.005*"also" + 0.005*"company"')]